In [23]:
from aiida import load_profile
load_profile()

Profile<uuid='468a9d4d0812492ba52462f2aca53e23' name='presto'>

In [24]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

#structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

In [25]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [26]:
from aiida.orm import load_code
code = load_code("janus@localhost")

In [27]:
from aiida.orm import Str
inputs = {
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "code": code,
    "arch": model.architecture,
    "precision": Str("float64"),
    "struct": structure,
    "model": model,
    "device": Str("cpu"),
}

In [28]:
from aiida.plugins import CalculationFactory
geomoptCalc = CalculationFactory("mlip.opt")
descriptorsCalc = CalculationFactory("mlip.descriptors")

In [29]:
from aiida_workgraph import WorkGraph

wg = WorkGraph("Geometry Optimisation")
wg.inputs = inputs

geomopt_calc = wg.add_task(
    geomoptCalc,
    name="geomopt_calc",
)

# descriptors_calc = wg.add_task(
#     descriptorsCalc,
#     name="descriptors_calc",
# )


In [30]:
wg.tasks.geomopt_calc.inputs = wg.inputs

In [31]:
wg.outputs.results = wg.tasks.geomopt_calc.outputs.final_structure
wg.outputs.results_file = wg.tasks.geomopt_calc.outputs.final_structure

In [32]:
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'Geometr…

In [33]:
wg.run()

07/22/2025 10:03:05 AM <358723> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7365|WorkGraphEngine|continue_workgraph]: tasks ready to run: geomopt_calc
07/22/2025 10:03:07 AM <358723> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7365|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 7370
07/22/2025 10:03:42 AM <358723> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7365|WorkGraphEngine|update_task_state]: Task: geomopt_calc, type: CALCJOB, finished.
07/22/2025 10:03:42 AM <358723> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7365|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
07/22/2025 10:03:42 AM <358723> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7365|WorkGraphEngine|finalize]: Finalize workgraph.


In [34]:
wg_output = wg.outputs.results_file.value

wg_output

<StructureData: uuid: 5860849d-d0c0-4cc8-a555-80e6b45e1f49 (pk: 7379)>

In [37]:
wg_inputs = {
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "code": code,
    "arch": model.architecture,
    "precision": Str("float64"),
    "struct": wg_output,
    "model": model,
    "device": Str("cpu"),
}

In [ ]:
# ! verdi data core.structure  export 7361 > wg_output.xsf

In [35]:
from aiida.plugins import CalculationFactory

descriptorsCalc = CalculationFactory("mlip.descriptors")

In [40]:
from aiida_workgraph import WorkGraph

wg = WorkGraph("Descriptors")

descriptors_calc = wg.add_task(
    descriptorsCalc,
    name="descriptors_calc",
    **wg_inputs
)


In [41]:
wg.run()

07/22/2025 10:05:33 AM <358723> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7382|WorkGraphEngine|continue_workgraph]: tasks ready to run: descriptors_calc
07/22/2025 10:05:36 AM <358723> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7382|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 7386
07/22/2025 10:06:05 AM <358723> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7382|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
07/22/2025 10:06:05 AM <358723> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [7382|WorkGraphEngine|finalize]: Finalize workgraph.


In [45]:
wg.tasks.descriptors_calc.outputs.results_dict.value.get_dict()

{'pbc': [True, True, True],
 'cell': [[0.0, 2.815, 2.815], [2.815, 0.0, 2.815], [2.815, 2.815, 0.0]],
 'info': {'arch': 'mace_mp',
  'emissions': 4.208576394309e-06,
  'model_path': 'mlff.model',
  'system_name': 'aiida',
  'mace_mp_descriptor': -0.0056343183056331},
 'masses': [22.98976928, 35.45],
 'numbers': [11, 17],
 'positions': [[0.0, 0.0, 0.0], [2.815, 0.0, 0.0]]}